In [1]:
!pip install -q pypdf chromadb sentence-transformers groq langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0

In [2]:
!pip install langchain-groq


In [ ]:
import os
import chromadb
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from dataclasses import dataclass
from IPython.display import display, Markdown

# Set up Groq AI client
groq_client = ChatGroq(api_key="gsk_19RFbIOTiZEEMZlOTuOEWGdyb3FYd1M9gff6CViZ93d8lCR*****", model_name="llama-3.1-8b-instant")

# Function to load and process PDFs from a folder
def load_pdfs_from_folder(folder_path):
    all_texts = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(folder_path, file_name)
            reader = PdfReader(file_path)
            text = "".join(page.extract_text() or "" for page in reader.pages)
            all_texts.append(text)
    return all_texts

def split_text_into_chunks(texts, chunk_size=500):
    chunks = []
    for text in texts:
        chunks.extend([text[i:i+chunk_size] for i in range(0, len(text), chunk_size)])
    return chunks

# Ask user for folder input
folder_path = input("Enter the path of the folder containing PDFs: ")
if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
    print("Invalid folder path. Please enter a valid path.")
    exit()

# Process all PDFs in the folder
pdf_texts = load_pdfs_from_folder(folder_path)
chunks = split_text_into_chunks(pdf_texts)

# Vectorize the text
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(chunks)

# Store embeddings in ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="pdf_documents")

for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    collection.add(documents=[chunk], embeddings=[embedding.tolist()], ids=[str(i)])

@dataclass
class RAGState:
    query: str
    context: str = ""
    response: str = ""

# Retrieve relevant text chunks
def retrieve_relevant_chunks(state):
    query = state.query.lower()
    query_embedding = embedding_model.encode([query])[0]
    results = collection.query(query_embeddings=[query_embedding.tolist()], n_results=3)

    if results and results['documents']:
        return RAGState(query=query, context="\n".join(results['documents'][0]))
    return RAGState(query=query, response="I'm not aware of what you're asking.")

# Generate response
def generate_response(state):
    query = state.query
    context = state.context if state.context else "I'm not aware of what you're asking."
    prompt = f"""
    You are an AI chatbot that answers questions based on the uploaded PDFs.
    If the question is unrelated to the documents, say: "I'm not aware of what you're asking."

    Context:
    {context}

    Question: {query}
    Answer:
    """
    response = groq_client.invoke(prompt)
    return RAGState(query=query, response=response)

# Define LangGraph Workflow
workflow = StateGraph(RAGState)
workflow.add_node("retrieve", retrieve_relevant_chunks)
workflow.add_node("generate", generate_response)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)
graph = workflow.compile()

# Chat function
def chat_with_pdfs():
    print("Welcome to the PDF Chatbot. Type 'exit' to quit.")
    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit", "q"]:
            break

        result = graph.invoke(RAGState(query=query))
        state = RAGState(**result)

        if isinstance(result['response'], str):
            print(f"Bot: {result['response']}")
        else:
            print(f"Bot: {result['response'].content}")

# Run the chatbot
chat_with_pdfs()


Enter the path of the folder containing PDFs: /content/KB


Welcome to the PDF Chatbot. Type 'exit' to quit.
You: suggest me places to see in ayodhya
Bot: Based on the uploaded PDF document "Ayodhya: The Sacred City", here are some suggested places to see in Ayodhya:

1. **Ram Janmabhoomi Temple**: The most famous religious site in Ayodhya, believed to be the birthplace of Lord Rama.
2. **Hanuman Garhi**: A prominent temple dedicated to Lord Hanuman, featuring a steep climb of 76 steps. Devotees visit to seek blessings before heading to Ram Janmabhoomi.
3. **Kanak Bhawan**: A temple known for its stunning idols of Lord Rama and Goddess Sita adorned with gold ornaments.
4. **Sarayu River**: A holy river where thousands of devotees take a holy dip on Kartik Purnima.

Additionally, Ayodhya Airport, improved rail and road networks, and new accommodations including luxury hotels and resorts are also emerging, making it easier for tourists to visit and explore the city.
You: give me 3 days plan to visit praygraj
Bot: Based on the provided information